In [1]:
import os
import sys
import math
import time
import glob 

import multiprocessing as mp
import numpy as np
import scipy.sparse as sp
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
%matplotlib inline

from liblibra_core import *
import util.libutil as comn
from libra_py import units as units
from libra_py import data_conv, data_stat, data_outs, data_read
import libra_py.workflows.nbra.decoherence_times as decoherence_times
import libra_py.workflows.nbra.step4 as step4
import libra_py.workflows.nbra.ann as ann

/home/98722002/miniconda3/envs/libra/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: to-Python converter for std::vector<std::vector<int, std::allocator<int> >, std::allocator<std::vector<int, std::allocator<int> > > > already registered; second conversion method ignored.
  return f(*args, **kwds)
/home/98722002/miniconda3/envs/libra/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: to-Python converter for boost::python::detail::container_element<std::vector<std::vector<int, std::allocator<int> >, std::allocator<std::vector<int, std::allocator<int> > > >, unsigned long, boost::python::detail::final_vector_derived_policies<std::vector<std::vector<int, std::allocator<int> >, std::allocator<std::vector<int, std::allocator<int> > > >, false> > already registered; second conversion method ignored.
  return f(*args, **kwds)
/home/98722002/miniconda3/envs/libra/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: to-Python converter for std::vector<std::vector<float,

In [2]:
%matplotlib inline
#notebook
#figure = plt.figure(num=None, figsize=(3.21, 2.41), dpi=300, edgecolor='black', frameon=True)        

def stretched_compressed(t, tau, beta):
    return np.exp( -np.power(( t/tau ),beta) )

for c, scheme in enumerate(['FSSH', 'IDA','mSDM']):
    figure = plt.figure(num=None, figsize=(3.21, 2.41), dpi=600, edgecolor='black', frameon=True)
    print(F'------------{scheme}----------')
    #plt.subplot(1,2,c+1)
    folders = glob.glob(F'./results/namd_regular_electron_only_sp-5/_start_s10_{scheme}_batch*', recursive = True)
    #print(folders)
    taus = []
    betas = []
    
    for folder in folders:
        
        print("Folder =", folder)
        sh_pop = np.loadtxt(folder+'/SH_pop.txt')
        sh_pop_hot_state = sh_pop[:,10]
        md_time = np.arange(0, sh_pop_hot_state.shape[0], 1.0)
        popt, pcov = curve_fit( stretched_compressed, md_time, sh_pop_hot_state, 
                                   bounds=([0.0, 0.0],[np.inf, np.inf]))
        tau, beta = popt
        # Computing the R-squared
        residuals  = sh_pop_hot_state - stretched_compressed(md_time, *popt)
        ss_res     = np.sum(residuals**2)
        ss_tot     = np.sum((sh_pop_hot_state - np.mean(sh_pop_hot_state))**2)
        r_squared  = 1.0 - (ss_res / ss_tot)
        print('tau:', tau, ' beta:', beta,' R2:', r_squared)
        
        if r_squared>0.01:
            taus.append(tau)
            betas.append(beta)
            plt.plot(md_time, sh_pop_hot_state,  linewidth=1, color = 'green')
            
    ave_tau = np.average(np.array(taus))
    ave_beta = np.average(np.array(betas))
    # Compute the erro bars
    # The confidence interval
    Z = 1.96
    N = len(taus)
    s = np.std(np.array(taus))
    error_bar = Z*s/np.sqrt(N)
    
######################################################################################################################        
    
    
    folders2 = glob.glob(F'./results/namd_regular_electron_only_sp-10/_start_s10_{scheme}_batch*', recursive = True)
    #print(folders2)
    taus2 = []
    betas2 = []    
    for folder in folders2:
        
        #print("Folder =", folder)
        sh_pop = np.loadtxt(folder+'/SH_pop.txt')
        sh_pop_hot_state = sh_pop[:,10]
        md_time2 = np.arange(0, sh_pop_hot_state.shape[0], 1.0)
        popt, pcov = curve_fit( stretched_compressed, md_time2, sh_pop_hot_state, 
                                   bounds=([0.0, 0.0],[np.inf, np.inf]))
        tau, beta = popt
        # Computing the R-squared
        residuals  = sh_pop_hot_state - stretched_compressed(md_time2, *popt)
        ss_res     = np.sum(residuals**2)
        ss_tot     = np.sum((sh_pop_hot_state - np.mean(sh_pop_hot_state))**2)
        r_squared  = 1.0 - (ss_res / ss_tot)
        print('tau:', tau, ' beta:', beta,' R2:', r_squared)
        
        if r_squared>0.01:
            taus2.append(tau)
            betas2.append(beta)
            plt.plot(md_time2, sh_pop_hot_state,  linewidth=1, color = "mediumorchid")
            
    ave_tau2 = np.average(np.array(taus2))
    ave_beta2 = np.average(np.array(betas2))
    # Compute the erro bars
    # The confidence interval
    Z = 1.96
    N = len(taus2)
    s = np.std(np.array(taus2))
    error_bar = Z*s/np.sqrt(N)
    
#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@2    
    
  #  folders3 = glob.glob(F'./results/namd_regular_hole_only_mb-Hvib_ci/_start_s10_{scheme}_batch*', recursive = True)
  #  taus3 = []
  #  betas3 = []    
  #  for folder in folders3:
  #      
  #      print("Folder =", folder)
  #      sh_pop = np.loadtxt(folder+'/SH_pop.txt')
  #      sh_pop_hot_state = sh_pop[:,10]
  #      md_time3 = np.arange(0, sh_pop_hot_state.shape[0], 1.0)
  #      popt, pcov = curve_fit( stretched_compressed, md_time3, sh_pop_hot_state, 
  #                                 bounds=([0.0, 0.0],[np.inf, np.inf]))
  #      tau, beta = popt
  #      # Computing the R-squared
  #      residuals  = sh_pop_hot_state - stretched_compressed(md_time3, *popt)
  #      ss_res     = np.sum(residuals**2)
  #      ss_tot     = np.sum((sh_pop_hot_state - np.mean(sh_pop_hot_state))**2)
  #      r_squared  = 1.0 - (ss_res / ss_tot)
  #      print('tau:', tau, ' beta:', beta,' R2:', r_squared)
  #      
  #      if r_squared>0.01:
  #          taus3.append(tau)
  #          betas3.append(beta)
  #          plt.plot(md_time3, sh_pop_hot_state,  linewidth=1, color = "skyblue")
  #          
  #  ave_tau3 = np.average(np.array(taus3))
  #  ave_beta3 = np.average(np.array(betas3))
  #  # Compute the erro bars
  #  # The confidence interval
  #  Z = 1.96
  #  N = len(taus3)
  #  s = np.std(np.array(taus3))
  #  error_bar = Z*s/np.sqrt(N)
  #  
#@#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@2    
  # 
  #  folders4 = glob.glob(F'./results/namd_regular_electron_only_mb-Hvib_ci/_start_s10_{scheme}_batch*', recursive = True)
  #  taus4 = []
  #  betas4 = []    
  #  for folder in folders4:
  #      
  #      print("Folder =", folder)
  #      sh_pop = np.loadtxt(folder+'/SH_pop.txt')
  #      sh_pop_hot_state = sh_pop[:,10]
  #      md_time4 = np.arange(0, sh_pop_hot_state.shape[0], 1.0)
  #      popt, pcov = curve_fit( stretched_compressed, md_time4, sh_pop_hot_state, 
  #                                 bounds=([0.0, 0.0],[np.inf, np.inf]))
  #      tau, beta = popt
  #      # Computing the R-squared
  #      residuals  = sh_pop_hot_state - stretched_compressed(md_time4, *popt)
  #      ss_res     = np.sum(residuals**2)
  #      ss_tot     = np.sum((sh_pop_hot_state - np.mean(sh_pop_hot_state))**2)
  #      r_squared  = 1.0 - (ss_res / ss_tot)
  #      print('tau:', tau, ' beta:', beta,' R2:', r_squared)
  #      
  #      if r_squared>0.01:
  #          taus4.append(tau)
  #          betas4.append(beta)
  #          plt.plot(md_time4, sh_pop_hot_state,  linewidth=1, color = "skyblue")
  #          
  #  ave_tau4 = np.average(np.array(taus4))
  #  ave_beta4 = np.average(np.array(betas4))
  #  # Compute the erro bars
  #  # The confidence interval
  #  Z = 1.96
  #  N = len(taus4)
  #  s = np.std(np.array(taus4))
  #  error_bar = Z*s/np.sqrt(N)
    
#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@    
  
#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@    
  
    
    print(F'The timescales: {ave_tau}+-{error_bar} average over {len(taus)} samples')
    # Plot average fit
    #plt.plot(md_time, stretched_compressed(md_time, ave_tau-error_bar, ave_beta), ls='--',linewidth=3, color='red')
    plt.plot(md_time, stretched_compressed(md_time, ave_tau,           ave_beta), ls='-',  linewidth=3,label='5', color='darkcyan')
    plt.plot(md_time2, stretched_compressed(md_time2, ave_tau2,           ave_beta2), ls='-',  linewidth=3,label='10', color='rebeccapurple')
    #plt.plot(md_time3, stretched_compressed(md_time3, ave_tau3,           ave_beta3), ls='-',  linewidth=3,label='c$_i$ $h^+$', color='blue')
    #plt.plot(md_time4, stretched_compressed(md_time4, ave_tau4,           ave_beta4), ls='-',  linewidth=3,label='c$_i$ $e^-$', color='red')

    #plt.plot(md_time, stretched_compressed(md_time, ave_tau+error_bar, ave_beta), ls='--', linewidth=3, color='red')
        
    plt.legend(fontsize=6, ncol=1, loc='upper right')
    plt.title(scheme)
    plt.xlabel('Time, fs')
    plt.ylabel('Population')
    plt.tight_layout()
    plt.savefig(F'./results/{scheme}electron_hole_only-mb-sd.png', dpi=600)
    plt.show() 

------------FSSH----------
The timescales: nan+-nan average over 0 samples


/home/98722002/miniconda3/envs/libra/lib/python3.6/site-packages/numpy/lib/function_base.py:380: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/home/98722002/miniconda3/envs/libra/lib/python3.6/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/98722002/miniconda3/envs/libra/lib/python3.6/site-packages/numpy/core/_methods.py:234: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/home/98722002/miniconda3/envs/libra/lib/python3.6/site-packages/numpy/core/_methods.py:195: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/home/98722002/miniconda3/envs/libra/lib/python3.6/site-packages/numpy/core/_methods.py:226: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


NameError: name 'md_time' is not defined

<Figure size 1926x1446 with 0 Axes>